In [1]:

# coding: utf-8

# In[ ]:


from __future__ import print_function

import sys, os
import numpy as np
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
os.environ['XILINX_XRT'] = '/usr'
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))
    print("Start of \"" + sys.argv[0] + "\"")
    
    # Overlay
    ol = Overlay("mq_ntt.bit")
    ipNTT = ol.mq_NTT_0

    NTT_in = open("mq_ntt_in.txt", "r+")

    # Memory allocate
    inBuffer0 = allocate(shape=(1024,), dtype=np.int16)
    
    NTT_in.seek(0)
    for i in range(1024):
        line = NTT_in.readline()
        inBuffer0[i] = int(line)
        print(f"input data: {inBuffer0[i]:d}")
    NTT_in.close()
    
    timeKernelStart = time()
    # ============= Set Kernel Argument ==============
    # 0x00 : Control signals
    #        bit 0  - ap_start (Read/Write/COH)
    #        bit 1  - ap_done (Read/COR)
    #        bit 2  - ap_idle (Read)
    #        bit 3  - ap_ready (Read/COR)
    #        bit 7  - auto_restart (Read/Write)
    #        bit 9  - interrupt (Read)
    #        others - reserved
    # 0x04 : Global Interrupt Enable Register
    #        bit 0  - Global Interrupt Enable (Read/Write)
    #        others - reserved
    # 0x08 : IP Interrupt Enable Register (Read/Write)
    #        bit 0 - enable ap_done interrupt (Read/Write)
    #        bit 1 - enable ap_ready interrupt (Read/Write)
    #        others - reserved
    # 0x0c : IP Interrupt Status Register (Read/COR)
    #        bit 0 - ap_done (Read/COR)
    #        bit 1 - ap_ready (Read/COR)
    #        others - reserved
    # 0x10 : Data signal of a
    #        bit 31~0 - a[31:0] (Read/Write)
    # 0x14 : Data signal of a
    #        bit 31~0 - a[63:32] (Read/Write)
    # 0x18 : reserved
    # 0x1c : Data signal of logn
    #        bit 31~0 - logn[31:0] (Read/Write)
    
    logn = 10
    ipNTT.write(0x1C, logn) # Coefficient

    # ================ Data Transfer =================
    ipNTT.write(0x10, inBuffer0.device_address) # 0x10 bit 31:0
    ipNTT.write(0x14, inBuffer0.device_address) # 0x14 bit 63:32

    # =================== Start ======================
    ipNTT.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
    while (ipNTT.read(0x00) & 0x4) == 0x0:
        continue
    timeKernelEnd = time()
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
    
    # ================== Compare =====================
    error_cnt = 0
    NTT_out = open("mq_ntt_out.txt", "r+")
    NTT_out.seek(0)
    for i in range(1024):
        line = NTT_out.readline()
        golden = int(line)
        if inBuffer0[i] != golden:
            print(f"Failed, expect value: {golden:d}, got value: {inBuffer0[i]:d}")
            error_cnt += 1
        #else:
            #print(f"Calculate value: {inBuffer0[i]:.17f}")
    NTT_out.close()
    if error_cnt == 0:
        print("Test Passed!!")
    else:
        print("Test Failed!!")

    print("============================")
    print("Exit process")

Entry: /usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py"
input data: 12264
input data: 12177
input data: 12177
input data: 12225
input data: 12099
input data: 15
input data: 337
input data: 12057
input data: 12207
input data: 12193
input data: 12191
input data: 12203
input data: 12147
input data: 106
input data: 90
input data: 12208
input data: 11856
input data: 12255
input data: 12161
input data: 12257
input data: 120
input data: 120
input data: 181
input data: 12281
input data: 12261
input data: 4
input data: 182
input data: 85
input data: 262
input data: 125
input data: 307
input data: 12082
input data: 126
input data: 12214
input data: 115
input data: 53
input data: 12180
input data: 12225
input data: 12221
input data: 202
input data: 36
input data: 12248
input data: 12103
input data: 12287
input data: 131
input data: 158
input data: 12271
input d